<div style="line-height:1.2;">

<h1 style="color:#BF66F2; margin-bottom: 0.3em;">Feedforward Neural Networks in PyTorch 3 </h1>

<h4 style="margin-top: 0.3em; margin-bottom: 1em;">Convert two models (PyTorch and TensorFlow) into ONNX format and use ONNX Runtime to perform inference. </h4>

<div style="line-height:1.4; margin-bottom: 0.5em;">
    <h3 style="color: lightblue; display: inline; margin-right: 0.5em;">Keywords:</h3> 
    torchsummary + tf2onnx + onnxruntime + TensorSpec
</div>

</div>


In [9]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim

import tensorflow as tf

import tf2onnx
import onnxruntime

from torchsummary import summary

In [2]:
%%script echo Skip, since already installed
!pip install torchsummary

Skip, since already installed


In [3]:
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(2, 1)

    def forward(self, x):
        return self.fc(x)

model = SimpleModel()

## Convert to ONNX format
dummy_input = torch.randn(1, 2)
torch.onnx.export(model, dummy_input, "./data/simple_model.onnx")

### Inference with ONNX Runtime
ort_session = onnxruntime.InferenceSession("./data/simple_model.onnx")
input_name = ort_session.get_inputs()[0].name
output_name = ort_session.get_outputs()[0].name

============= Diagnostic Run torch.onnx.export version 2.0.1+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



<h3 style="color:#BF66F2 "> Recap: torchsummary </h3>
<div style="margin-top: -8px;">
Show layers, output shapes, and the number of parameters (similar to Keras' model.summary()).

In [8]:
summary(model, input_size=(2,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                    [-1, 1]               3
Total params: 3
Trainable params: 3
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [4]:
# Create random input
x_input = np.random.rand(1, 2).astype(np.float32)
# Run inference
result = ort_session.run([output_name], {input_name: x_input})

In [6]:
model = tf.keras.Sequential([tf.keras.layers.Dense(1, input_shape=[2])])

# Convert to ONNX format
model.save("simple_model")
spec = (tf.TensorSpec((1, 2), tf.float32, name="input"),)
converted_model_path = tf2onnx.convert.from_keras(model, input_signature=spec, output_path="./data/simple_model.onnx")

# Inference with ONNX Runtime
ort_session = onnxruntime.InferenceSession("./data/simple_model.onnx")
input_name = ort_session.get_inputs()[0].name
output_name = ort_session.get_outputs()[0].name

2023-10-07 21:11:05.175687: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2023-10-07 21:11:05.175789: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: hpmint
2023-10-07 21:11:05.175805: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: hpmint
2023-10-07 21:11:05.175873: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 390.157.0
2023-10-07 21:11:05.175912: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 390.157.0
2023-10-07 21:11:05.175931: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 390.157.0


INFO:tensorflow:Assets written to: simple_model/assets


INFO:tensorflow:Assets written to: simple_model/assets
2023-10-07 21:11:06.055917: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-10-07 21:11:06.056112: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2023-10-07 21:11:06.092838: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-10-07 21:11:06.093035: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session


In [7]:
# Create random input
x_input = np.random.rand(1, 2).astype(np.float32)
# Run inference
result = ort_session.run([output_name], {input_name: x_input})

In [8]:
result

[array([[-0.583429]], dtype=float32)]